In [150]:
from sympy import symbols, Eq, solve

# Define the symbols
R1, R2 = symbols('R1 R2')

# Known values
Vin = 10  # Input voltage in volts
Vout = 2  # Desired output voltage in volts
I = 0.001 # Desired output current in amps

# Define the equations
equation1 = Eq(Vout, Vin * (R2 / (R1 + R2)))
equation2 = Eq(I, Vin / (R1 + R2))
ratio = R2 / (R1 + R2)

# Solve the equations
solution = solve((equation1, equation2), (R1, R2))

solution

{R1: 8000.00000000000, R2: 2000.00000000000}

In [151]:
from eseries import find_nearest, E24

R1 = (find_nearest(E24, solution[R1]))
R2 = (find_nearest(E24, solution[R2]))



### Step 1: solver

Takes in any variables that seem useful and a set of equations and returns a solution for the variables.

### Step 2: component picker

Finds the nearest component values to the solution

### Step 3: checker

Checks the proposed values against the equations to see if it is correct

In [167]:
from sympy import symbols, Eq, solve

import rich

# Define all symbols
R1, R2, Vin, Vout, I, ratio, r_total = symbols('R1 R2 Vin Vout I ratio r_total')

# Define the equations
equation1 = Eq(Vout, Vin * (R2 / (R1 + R2)))
equation2 = Eq(I, Vin / (R1 + R2))
equation3 = Eq(ratio, R2 / (R1 + R2))
equation4 = Eq(r_total,R1 + R2)


# Function to solve the equations based on specified values
def solve_circuit(values):
    substituted_eqs = [eq.subs(values) for eq in [equation1, equation2, equation3, equation4]]

    # Determine which variables are still unknown
    unknowns = [var for var in [R1, R2, Vin, Vout, I, ratio, r_total] if var not in values]

    # Solve the equations for the unknowns
    solution = solve(substituted_eqs, unknowns)
    return {str(var): sol for var, sol in zip(unknowns, solution[0])}
    # return solution



# Example usage:

# Case: Solving for unknowns when R1 and R2 are specified
print(solve_circuit({ratio: 0.5, r_total: 2}))
print(solve_circuit({ratio: 0.2, r_total: 2, Vin: 10}))

# solve_circuit({I: 0.001, r_total: 2, Vin: 10})

print(solve_circuit({I: 0.001, r_total: 2, ratio: 0.1}))

print(solve_circuit({I: 0.001, Vout: 2, Vin: 10}))






{'R1': (1.00000000000000, 1.00000000000000, 2.0*Vout, Vout, Vout)}
{'R1': (1.60000000000000, 0.400000000000000, 2.00000000000000, 5.00000000000000)}
{'R1': (1.80000000000000, 0.200000000000000, 0.00200000000000000, 0.000200000000000000)}
{'R1': (8000.00000000000, 2000.00000000000, 0.200000000000000, 10000.0000000000)}


In [169]:
#solve

solution = solve_circuit({I: 0.001, Vout: 2, Vin: 10})

# Find the nearest E24 values
r1 = eseries.find_nearest(eseries.E24, solution["R1"])
r2 = eseries.find_nearest(eseries.E24, solution["R2"])

# Check the results are within tolerance
tolerance = 0.05

# proposed_solution = solve_circuit({R1: 10, R2: 10, Vin: 10})

# proposed_solution



TypeError: unsupported operand type(s) for /: 'tuple' and 'float'

In [62]:
from sympy import symbols, solve_univariate_inequality

inequality1 = R1 > 10
inequality2 = R1 < 11
solution1 = solve_univariate_inequality(inequality1, R1)
solution2 = solve_univariate_inequality(inequality2, R1)



(10 < R1) & (R1 < oo)

In [25]:
import unyts as ut

# ut.volts(10, 'V')

resistance = 100 * ut.ohm

resistance

100_Ω

In [36]:
import pint

ureg = pint.UnitRegistry()

(ureg.millimeter / ureg.kilometer)


<Unit('millimeter / kilometer')>

In [ ]:
#ato code example

#### Vdiv class ####
r1: Resistance = 3R | 3ohm
r2: Resistance = 4K
vout: voltage = 10V
i(amps)
rtotal(ohms)

vout = vin * (r2 / (r1 + r2))

i = vin / (r1 + r2)

rtotal = r1 + r2

r_top.value = r1
r_bt.value = r2


#### Instance of Vdiv class ####

vdiv = new Vdiv

vdiv.vin = 12V
vdiv.vout = 10V
# vdiv.i = 0.001A
vdiv.rtotal = 1000Kohms


component Resistor:
    signal p1 ~ pin 1
    signal p2 ~ pin 2

    resistance = unknown ohms

module Vdiv:
    r1 = new Resistor 
    r2 = new Resistor

    signal in
    signal out
    signal gnd

    variable current = in.current
    variable rtotal = unknown ohms


    # equations
    out.voltage:  = in.voltage * (r2.resistance / (r1.resistance + r2.resistance))
    in.current = in.voltage / (r1.resistance + r2.resistance)
    rtotal = r1.resistance + r2.resistance
    ratio Ohms = r2.resistance / (r1.resistance + r2.resistance) kohm


vdiv = new Vdiv
vdiv.current = 0.01A
vdiv.in.current = 0.01A

vdiv.ratio = 0.5
vdiv.rtotal = 1000

In [28]:
import re

def parse_unit(unit_str):
    # Define prefixes and their multipliers
    prefixes = {
        'm': 1e-3,  # milli
        'u': 1e-6,  # micro
        'k': 1e3,   # kilo
        'M': 1e6,   # Mega
        # ... add more prefixes as needed
    }

    # Define the regex pattern for parsing
    pattern = r'(\d*\.?\d+)([a-zA-Z]*)'

    # Search for matches using the regex pattern
    match = re.match(pattern, unit_str)
    if not match:
        raise ValueError(f"Invalid unit format: {unit_str}")

    # Extract value and unit from the match
    value_str, unit_with_prefix = match.groups()

    # Convert the value string to a float
    value = float(value_str)

    # Check if the unit has a prefix and separate it
    prefix = ''
    for p in prefixes:
        if unit_with_prefix.startswith(p):
            prefix = p
            break

    # Remove the prefix from the unit if it exists
    unit = unit_with_prefix[len(prefix):] if prefix else unit_with_prefix

    # Apply the multiplier if a prefix was found
    if prefix in prefixes:
        value *= prefixes[prefix]

    return value, unit

# Example usage
value, unit = parse_unit("2.5mA")
print(f"Value: {value} A, Unit: {unit}")




Value: 0.0025 A, Unit: A
